In [1]:
from maxpooling import maxPooling 
from softmax import softMax
from Flatten import Flatten
from Batchnorm import BatchNorm
from Convolution_Function import Convolution 
from Dropout import Layer_Dropout 
from FullyConnectedLayer_Function import FullyConected
from Convolution_Function import ReLU 
import numpy as np

In [2]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = np.array(train_images)
train_labels = np.array(train_labels)
test_images = np.array(test_images)
test_labels = np.array(test_labels)

test_images.shape

(10000, 28, 28)

In [3]:
conv_layer = Convolution(1, 8, 3) # 26x26x8 
relu = ReLU()
#batchnorm_layer = BatchNorm()
pooling_layer = maxPooling(2, 2)  # 13x13x8 
flatten_layer = Flatten()
flc_layer1 = FullyConected(13*13*8, 128)
dropout_layer1 = Layer_Dropout(0.5)
flc_layer2 = FullyConected(128, 32)
dropout_layer2 = Layer_Dropout(0.5)
softmax_layer = softMax(32,10)

In [ ]:
class CNN:
    def __init__(self, layers):
        self.layers = layers
    
    def forward(self, X):
        """
        Forward pass through the CNN
        X: input image of shape (height, width, channels)
        """
        output = X
        print("Original: ", output.shape)

        # Convolutional layer
        output = self.layers[0].forward(output)  # Convolution: (28, 28, 1) -> (26, 26, 8)
        print("Conv layer:", output.shape)

        # ReLU activation
        output = self.layers[1].forward(output)  # ReLU
        print("ReLU layer: ", output.shape)

        # Max pooling
        output = self.layers[2].forward(output)  # MaxPooling: (26, 26, 8) -> (13, 13, 8)
        print("Max pooling", output.shape)

        # Flatten
        output = self.layers[3].forward(output)  # Flatten: (13, 13, 8) -> (1352, 1)
        print("Flatten", output.shape)
        
        # Fully connected layer 1
        output = self.layers[4].forward(output)  # FC1: (1352,) -> (128,)
        print("Fully 1: ", output.shape)
        # ReLU activation
        output = np.maximum(0, output)  # ReLU
        print("ReLU: ", output.shape)
        # Dropout layer 1
        self.layers[5].forward(output)  # Dropout: (128,)
        output = self.layers[5].output  # Apply dropout mask
        print("Dropout: ", output.shape)
        
        # Fully connected layer 2
        output = self.layers[6].forward(output)  # FC2: (128,) -> (32,)
        
        # ReLU activation
        output = np.maximum(0, output)  # ReLU
        
        # Dropout layer 2
        self.layers[7].forward(output)  # Dropout: (32,)
        output = self.layers[7].output  # Apply dropout mask
        
        # Softmax
        output = self.layers[8].forward(output)  # Softmax: (32,) -> (10,)
        print("Softmax: ", output.shape)

        
        return output
    
    def backward(self, dY, learning_rate):
        """
        Backward pass through the CNN
        dY: gradient of loss with respect to softmax output
        learning_rate: learning rate for weight updates
        """
        # Softmax backward
        dY = self.layers[8].backprop(dY, learning_rate)
        print("Softmax backward")

        # Dropout layer 2 backward
        dY = self.layers[7].backward(dY)
        dY = np.maximum(0, dY)  # ReLU backward
        
        # FC2 backward
        dY = self.layers[6].backprop(dY, learning_rate)
        dY = np.maximum(0, dY)  # ReLU backward (simple version)
        
        # Dropout layer 1 backward
        dY = self.layers[5].backward(dY)
        dY = np.maximum(0, dY)  # ReLU backward
        
        # FC1 backward
        dY = self.layers[4].backprop(dY, learning_rate)
        
        # Flatten backward
        dY = self.layers[3].backward(dY)
        
        # MaxPooling backward
        dY = self.layers[2].backprop(dY)
        
        # ReLU backward
        dY = self.layers[1].backward(dY)
        
        # Convolution backward
        dY = self.layers[0].backprop(dY, learning_rate)
        
        return dY
    
    def train(self, train_images, train_labels, epochs, batch_size, learning_rate):
        """
        Train the CNN model
        train_images: training images of shape (num_samples, 28, 28)
        train_labels: training labels of shape (num_samples,)
        epochs: number of training epochs
        batch_size: batch size for training
        learning_rate: learning rate for optimization
        """
        num_samples = train_images.shape[0]
        
        for epoch in range(epochs):
            total_loss = 0
            num_batches = 0
            
            # Shuffle the data
            indices = np.arange(num_samples)
            np.random.shuffle(indices)
            train_images_shuffled = train_images[indices]
            train_labels_shuffled = train_labels[indices]
            
            # Iterate through batches
            for batch_idx in range(0, num_samples, batch_size):
                end_idx = min(batch_idx + batch_size, num_samples)
                batch_images = train_images_shuffled[batch_idx:end_idx]
                batch_labels = train_labels_shuffled[batch_idx:end_idx]
                
                batch_loss = 0
                
                # Process each sample in the batch
                for img, label in zip(batch_images, batch_labels):
                    # Normalize image
                    img_normalized = img.astype(np.float32) / 255.0
                    img_normalized = img_normalized[np.newaxis, :, :]  # Add channel dimension
                    
                    # Forward pass
                    output = self.forward(img_normalized)
                    
                    # Compute cross-entropy loss
                    output = np.clip(output, 1e-7, 1 - 1e-7)  # Avoid log(0)
                    loss = -np.log(output[label])
                    batch_loss += loss
                    
                    # Compute gradient of loss with respect to softmax output
                    dY = np.zeros_like(output)
                    dY[label] = -1 / output[label]
                    
                    # Backward pass
                    self.backward(dY, learning_rate)
                
                avg_batch_loss = batch_loss / (end_idx - batch_idx)
                total_loss += avg_batch_loss
                num_batches += 1
                
                if num_batches % 10 == 0:
                    print(f"Epoch {epoch+1}, Batch {num_batches}: Loss = {avg_batch_loss:.4f}")
            
            avg_epoch_loss = total_loss / num_batches
            print(f"Epoch {epoch+1} completed - Average Loss: {avg_epoch_loss:.4f}")
    
    def predict(self, test_images):
        """
        Make predictions on test images
        test_images: test images of shape (num_samples, 28, 28)
        """
        predictions = []
        
        for img in test_images:
            # Normalize image
            img_normalized = img.astype(np.float32) / 255.0
            img_normalized = img_normalized[np.newaxis, :, :]  # Add channel dimension
            
            # Forward pass
            output = self.forward(img_normalized)
            
            # Get predicted class
            predicted_class = np.argmax(output)
            predictions.append(predicted_class)
        
        return np.array(predictions)

In [9]:
# Instantiate the CNN model
cnn_model = CNN([
    conv_layer,        # 0: Convolution
    relu,              # 1: ReLU
    pooling_layer,     # 3: MaxPooling
    flatten_layer,     # 4: Flatten
    flc_layer1,        # 5: FC1
    dropout_layer1,    # 6: Dropout1
    flc_layer2,        # 7: FC2
    dropout_layer2,    # 8: Dropout2
    softmax_layer      # 9: Softmax
])

# Train the model (example with small subset and few epochs for testing)
# Uncomment to run:
cnn_model.train(train_images[:100], train_labels[:100], epochs=2, batch_size=10, learning_rate=0.01)

# Make predictions on test set
# predictions = cnn_model.predict(test_images[:10])
# print("Predictions:", predictions)

Original:  (1, 28, 28)
Conv layer: (8, 26, 26)
ReLU layer:  (8, 26, 26)
Max pooling (8, 13, 13)
Flatten (1352,)
Fully 1:  (1, 128)
ReLU:  (1, 128)
Dropout:  (1, 128)
Softmax:  (10,)


ValueError: setting an array element with a sequence.